In [2]:
%pip install git+https://github.com/facebookresearch/CrypTen.git

  Cloning https://github.com/facebookresearch/CrypTen.git to c:\users\work\appdata\local\temp\pip-req-build-1vgdfp7i
  Resolved https://github.com/facebookresearch/CrypTen.git to commit 775868a02d6dac50774ce376a55b01fbd8bd85b6
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/CrypTen.git 'C:\Users\Work\AppData\Local\Temp\pip-req-build-1vgdfp7i'
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEA

In [4]:
%pip install tenseal

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.2 MB 7.4 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.2 MB 3.9 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.2 MB 3.9 MB/s eta 0:00:01
   --------------------------------- ------ 1.8/2.2 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 2.2 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import crypten
import tenseal as ts

print("CrypTen version:", crypten.__version__)
print("TenSEAL loaded successfully!")

ModuleNotFoundError: No module named 'crypten'

In [ ]:
import torch
import torch.nn as nn
import tenseal as ts
import time, os, psutil
import pandas as pd
import matplotlib.pyplot as plt

# --- Define PyTorch Model (same architecture) ---
class DeepNN(nn.Module):
    def __init__(self, input_dim):
        super(DeepNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 48),
            nn.ReLU(),
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

# --- Helpers ---
def measure_memory():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)  # MB

def plaintext_inference(model, x):
    start_mem = measure_memory()
    start = time.time()
    _ = model(x)
    end = time.time()
    end_mem = measure_memory()
    return (end - start) * 1000, (end_mem - start_mem)  # ms, MB

# ---------- TenSEAL Inference (with depth/scale control) ----------
def tenseal_inference(model, x):
    """
    Encrypted inference using CKKS with careful scale management:
      - Large initial scale (2**40)
      - Long modulus chain for ~10 rescale levels
      - ReLU -> (x * 0.25)^2  (keeps magnitude small)
      - Sigmoid -> 0.5 + 0.125 * x  (degree-1 proxy to preserve depth)
    """
    # 1) Context with deeper modulus chain to support many rescale steps
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=16384,  # more slots & noise budget
        coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 40, 40, 40, 60]  # ~8 rescale levels
    )
    context.global_scale = 2**40
    context.generate_galois_keys()

    # 2) Encrypt input (optionally shrink magnitude to help stability)
    x_plain = x.tolist()[0]
    enc_vec = ts.ckks_vector(context, x_plain)

    # ----- Activation proxies -----
    def relu_proxy(v):
        # tamed square: (x * 0.25)^2
        v = v.mul_plain(0.25)        # multiply by plaintext scalar (adds a level, small)
        v = v.square()               # ciphertext-ciphertext mult (adds a level)
        v.rescale_next()
        return v

    def sigmoid_proxy(v):
        # degree-1 proxy: 0.5 + 0.125 * x
        v = v.mul_plain(0.125)       # multiply by small scalar
        v.rescale_next()
        v = v.add_plain(0.5)
        return v

    # 3) Encrypted forward with explicit rescale after each mult
    def encrypted_forward(enc_v, model):
        for layer in model.model:
            if isinstance(layer, nn.Linear):
                W = layer.weight.detach().numpy()  # (out, in)
                b = layer.bias.detach().numpy()    # (out,)
                # enc_v.mm does ciphertext * plaintext-matrix (adds one mult level)
                enc_v = enc_v.mm(W.T)              # -> (out,)
                enc_v.rescale_next()
                enc_v = enc_v.add_plain(b.tolist())  # add doesn't consume a level
            elif isinstance(layer, nn.ReLU):
                enc_v = relu_proxy(enc_v)
            elif isinstance(layer, nn.Sigmoid):
                enc_v = sigmoid_proxy(enc_v)
            else:
                # No-ops for layers that aren't used here
                pass
        return enc_v

    start_mem = measure_memory()
    start = time.time()
    enc_out = encrypted_forward(enc_vec, model)
    end = time.time()
    end_mem = measure_memory()

    # Optional: decrypt to check (won't affect timings above)
    _ = enc_out.decrypt()

    return (end - start) * 1000, (end_mem - start_mem)  # ms, MB

# --- Run Experiments (TenSEAL vs Plaintext) ---
input_dims = [15, 50, 100]
results = []

for dim in input_dims:
    model = DeepNN(dim).eval()
    x = torch.randn(1, dim)

    # Plaintext baseline
    t_plain, m_plain = plaintext_inference(model, x)

    # TenSEAL encrypted
    t_tenseal, m_tenseal = tenseal_inference(model, x)

    results.append({
        "input_dim": dim,
        "plaintext_time_ms": t_plain,
        "tenseal_time_ms": t_tenseal,
        "plaintext_mem_mb": m_plain,
        "tenseal_mem_mb": m_tenseal
    })

results_df = pd.DataFrame(results)
print(results_df)

# --- Plotting (TenSEAL vs Plaintext) ---
def plot_performance_graphs(results_df_plot):
    plt.style.use('seaborn-v0_8-darkgrid')
    fig_width, fig_height = 10, 6
    os.makedirs('figures', exist_ok=True)

    # Time vs Input Size
    plt.figure(figsize=(fig_width, fig_height))
    plt.plot(results_df_plot['input_dim'], results_df_plot['plaintext_time_ms'], label='Plaintext NN', marker='o', color='blue')
    plt.plot(results_df_plot['input_dim'], results_df_plot['tenseal_time_ms'], label='TenSEAL Encrypted NN', marker='s', color='green')
    plt.xlabel('Input Dimension ($n$)')
    plt.ylabel('Inference Time (ms)')
    plt.title('Computation Time vs Input Dimension (Plain vs TenSEAL)')
    plt.legend()
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(True, which="both", ls="--", c='0.7')
    plt.tight_layout()
    plt.savefig('./figures/time_vs_input_size.png')
    plt.show()

    # Memory vs Input Size
    plt.figure(figsize=(fig_width, fig_height))
    plt.plot(results_df_plot['input_dim'], results_df_plot['plaintext_mem_mb'], label='Plaintext NN (Activations)', marker='o', color='blue')
    plt.plot(results_df_plot['input_dim'], results_df_plot['tenseal_mem_mb'], label='TenSEAL NN (Activations)', marker='s', color='green')
    plt.xlabel('Input Dimension ($n$)')
    plt.ylabel('Memory Usage (MB)')
    plt.title('Activation Memory Usage vs Input Dimension (Plain vs TenSEAL)')
    plt.legend()
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(True, which="both", ls="--", c='0.7')
    plt.tight_layout()
    plt.savefig('./figures/memory_vs_input_size.png')
    plt.show()

    # Relative Overhead vs Input Size
    plt.figure(figsize=(fig_width, fig_height))
    overhead_tenseal = ((results_df_plot['tenseal_time_ms'] - results_df_plot['plaintext_time_ms']) / results_df_plot['plaintext_time_ms']) * 100
    plt.plot(results_df_plot['input_dim'], overhead_tenseal, label='TenSEAL Overhead (%)', marker='s', color='orange')
    plt.xlabel('Input Dimension ($n$)')
    plt.ylabel('Relative Overhead (%)')
    plt.title('TenSEAL Relative Time Overhead vs Input Dimension')
    plt.legend()
    plt.xscale('log')
    plt.grid(True, which="both", ls="--", c='0.7')
    plt.tight_layout()
    plt.savefig('./figures/relative_overhead_vs_input_size.png')
    plt.show()

plot_performance_graphs(results_df)


ValueError: scale out of bounds